# Ludwig Time Series Forecasting

https://github.com/uber/ludwig

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time

In [2]:
matplotlib.rcParams['figure.figsize'] = (16, 9)
pd.options.display.max_columns = 999

## Load Dataset

In [3]:
df = pd.read_csv('../_datasets/hourly-weather-wind_speed.csv', parse_dates=[0], index_col='DateTime')
print(df.shape)
df.head()

(5000, 36)


,ts1,ts2,ts3,ts4,ts5,ts6,ts7,ts8,ts9,ts10,ts11,ts12,ts13,ts14,ts15,ts16,ts17,ts18,ts19,ts20,ts21,ts22,ts23,ts24,ts25,ts26,ts27,ts28,ts29,ts30,ts31,ts32,ts33,ts34,ts35,ts36
DateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-03-25 01:00:00,3.0,4.0,2.0,4.0,7.0,4.0,6.0,2.0,8.0,3.0,3.0,4.0,4.0,3.0,4.0,4.0,7.0,4.0,5.0,4.0,6.0,3.0,3.0,7.0,4.0,5.0,1.0,3.0,2.0,7.0,1.0,0.0,3.0,4.0,4.0,0.0
2017-03-25 02:00:00,3.0,4.0,3.0,5.0,6.0,1.0,7.0,2.0,5.0,1.0,2.0,4.0,5.0,4.0,4.0,4.0,8.0,3.0,3.0,1.0,3.0,3.0,2.0,6.0,4.0,6.0,2.0,3.0,2.0,6.0,1.0,0.0,5.0,4.0,4.0,0.0
2017-03-25 03:00:00,2.0,3.0,3.0,3.0,5.0,1.0,7.0,3.0,6.0,0.0,2.0,2.0,5.0,2.0,4.0,4.0,2.0,4.0,4.0,1.0,4.0,3.0,2.0,5.0,3.0,3.0,2.0,3.0,1.0,5.0,1.0,1.0,3.0,4.0,4.0,1.0
2017-03-25 04:00:00,2.0,3.0,3.0,4.0,2.0,4.0,3.0,1.0,1.0,0.0,2.0,3.0,10.0,2.0,4.0,5.0,5.0,3.0,4.0,2.0,4.0,2.0,2.0,5.0,4.0,3.0,2.0,2.0,1.0,5.0,1.0,1.0,6.0,0.0,0.0,1.0
2017-03-25 05:00:00,2.0,2.0,3.0,4.0,5.0,2.0,2.0,1.0,4.0,1.0,2.0,6.0,2.0,3.0,2.0,4.0,7.0,2.0,3.0,2.0,3.0,3.0,2.0,6.0,3.0,2.0,2.0,4.0,1.0,6.0,1.0,1.0,4.0,3.0,3.0,1.0


## Define Parameters

Make predictions for 24-hour period using a training period of four weeks.

In [4]:
dataset_name = 'Hourly Weather Wind Speed'
dataset_abbr = 'HWS'
model_name = 'Ludwig'
context_length = 24*7*4 # Four weeks
prediction_length = 24

## Define Error Metric

The seasonal variant of the mean absolute scaled error (MASE) will be used to evaluate the forecasts.

In [ ]:
def calc_sMASE(training_series, testing_series, prediction_series, seasonality=prediction_length):
    a = training_series.iloc[seasonality:].values
    b = training_series.iloc[:-seasonality].values
    d = np.sum(np.abs(a-b)) / len(a)
    errors = np.abs(testing_series - prediction_series)
    return np.mean(errors) / d

## Evaluating Ludwig

To evaluate Ludwig, forecasts will be generated for each time series. sMASE will be calculated for each individual time series, and the mean of all these scores will be used as the overall accuracy metric for Ludwig on this dataset.

### Prepare model definition file

In [ ]:
!touch ludwig.yaml

In [ ]:
config_str = """input_features:
    -
        name: {}
        type: timeseries

output_features:
""".format(dataset_abbr)

for i in range(prediction_length):
    config_str += """    -
        name: y{}
        type: numerical
""".format(i+1)

In [ ]:
with open("ludwig.yaml", "w+") as f:
    f.write(config_str)

### Prepare data

In [ ]:
df1 = df.iloc[-(context_length+prediction_length):]
df1_train = df1.iloc[:-prediction_length]
df1_test = df1.iloc[-prediction_length:]

In [ ]:
df2 = pd.DataFrame()

for i, col in enumerate(df1.columns):
    y_cols = ['y%s' % str(j+1) for j in range(prediction_length)]
    cols = [dataset_abbr] + y_cols
    
    train = df1_train[col].values
    test = df1_test[col].values
    train_str = ""
    for val in train:
        train_str += str(val) + " "
    train_str = train_str[:-1]
    
    vals = [train_str] + list(test)
    df_t = pd.DataFrame([vals], columns=cols, index=[i])
    df2 = df2.append(df_t)

In [ ]:
df2.to_csv('full.csv', index=False)

### Run Model

For this dataset and these parameters, the Ludwig model fails to complete training within an acceptable period of time

In [ ]:
!ludwig experiment --data_csv full.csv --model_definition_file ludwig.yaml